# macOS Userspace Type Library

## Compiler configuration

Include directories and macro definitions need to be configured for the source tree to successfullly parse. Specifically, macros such as `__arm64__` must be defined as certain headers are included conditionally by platform.

In [1]:
INCLUDE_DIRS = ["data/usr/include"]
COMPILER_FLAGS = ["-D__arm64__=1"]

## Target setup

For now, a subset of headers are selected to parse types from. In the future, this can be expanded to a wider selection of headers.

In [2]:
ROOT = "data/usr/include/"
PATH_QUERIES = [
    "mach/clock.h",
    "mach/mach_host.h",
    "mach/mach_port.h"
]

from glob import glob

SOURCE_PATHS = []
for p in PATH_QUERIES:
    SOURCE_PATHS += glob(ROOT + p)

## Type parsing

Parsing the types from source is relatively simple using the Clang type parser. The architecture and platform are currently hardcoded but can be easily made configurable later.

In [3]:
from binaryninja import *

target_arch = Architecture["aarch64"]
target_platform = Platform["mac-aarch64"]
type_parser = TypeParser["ClangTypeParser"]

types = {}
functions = {}

for path in SOURCE_PATHS:
    with open(path, "r") as f:
        source = f.read()

    (result, _) = type_parser.parse_types_from_source(
        source, path, target_platform, [], COMPILER_FLAGS, INCLUDE_DIRS
    )
    
    for t in result.types:
        types[t.name] = t.type
    
    for f in result.functions:
        functions[f.name] = f.type

## Library serialization

Finally, the type library needs to be constructed and serialized to disk. Note the addition of the leading underscore to symbol names to ensure function names match those in the binary.

In [4]:
OUTPUT_PATH = "out/demo.bntl"

typelib = TypeLibrary.new(target_arch, "libSystem.B.dylib")
typelib.add_platform(target_platform)

for t in types:
    typelib.add_named_type(t, types[t])
for f in functions:
    typelib.add_named_object(f"_{f}", functions[f])

typelib.finalize()
typelib.write_to_file(OUTPUT_PATH)